In [1]:
import sys
sys.path.insert(0, '../../src/')
import argparse
import pickle as pkl
import numpy as np
import torch
from torch.utils.data import DataLoader

from data_generator import train_val_test_split
from models import NonlinearSCI
from trainers import Trainer

np.random.seed(2020)
torch.manual_seed(2020)
torch.cuda.manual_seed(2020)
torch.cuda.manual_seed_all(2020)
torch.backends.cudnn.deterministic = True

/Users/a45329/miniconda/envs/zj63/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def main(args):
    with open('../../data/dataset_1.pkl', 'rb') as f:
        data = pkl.load(f)
    ndvi = [np.array([x[3] for x in data])]   # intervention
    nlcd = np.array([x[2] for x in data])     # confounder
    spatial_features = np.array([abs(x[0]-x[1]) for x in data])
    targets = np.array([x[4] for x in data])

    train_dataset, val_dataset, test_dataset = train_val_test_split(
        ndvi, nlcd, spatial_features, targets, train_size=0.6, val_size=0.2, test_size=0.2, shuffle=True
    )
    train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)
    dataloaders = {'train': train_dataloader, 'val': val_dataloader, 'test': test_dataloader}

    min_dist, max_dist = np.min(spatial_features), np.max(spatial_features)
    model = NonlinearSCI(
        num_interventions=1, 
        window_size=ndvi[0].shape[-1], 
        confounder_dim=nlcd.shape[-1], 
        f_network_type="dk_convnet", 
        g_network_type="dk_mlp", 
        unobserved_confounder=True, 
        nys_space=[[min_dist,max_dist]]
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optim = args.optim_name
    optim_params = {
        'lr': args.lr,
        'momentum': args.momentum
    }
    epochs, patience = args.n_epochs, args.patience
    trainer = Trainer(
        model=model, 
        data_generators=dataloaders, 
        optim=optim, 
        optim_params=optim_params, 
        device=device,
        epochs=epochs,
        patience=patience
    )
    trainer.train()

In [3]:
if __name__ == '__main__':
    arg_parser = argparse.ArgumentParser(description='f: DeepKriging with CNN, g: DeepKriging with MLP')
    arg_parser.add_argument('--batch_size', type=int, default=16)
    arg_parser.add_argument('--optim_name', type=str, default="sgd")
    arg_parser.add_argument('--lr', type=float, default=1e-6)
    arg_parser.add_argument('--momentum', type=float, default=0.99)
    arg_parser.add_argument('--n_epochs', type=int, default=1000)
    arg_parser.add_argument('--patience', type=int, default=10)
    args = arg_parser.parse_known_args()[0]
    main(args)

Training started:

Epoch 1/1000
Learning rate: 0.000001
2s for 12 steps - 152ms/step - loss 34.3768
Validation:
1s - loss 37.0611

Epoch 2/1000
Learning rate: 0.000001
2s for 12 steps - 143ms/step - loss 34.1603
Validation:
1s - loss 36.7019

Epoch 3/1000
Learning rate: 0.000001
2s for 12 steps - 147ms/step - loss 33.7310
Validation:
1s - loss 36.1510

Epoch 4/1000
Learning rate: 0.000001
2s for 12 steps - 147ms/step - loss 33.1369
Validation:
1s - loss 35.4198

Epoch 5/1000
Learning rate: 0.000001
2s for 12 steps - 145ms/step - loss 32.3612
Validation:
1s - loss 34.5278

Epoch 6/1000
Learning rate: 0.000001
2s for 12 steps - 154ms/step - loss 31.4537
Validation:
1s - loss 33.4817

Epoch 7/1000
Learning rate: 0.000001
2s for 12 steps - 147ms/step - loss 30.3853
Validation:
1s - loss 32.2931

Epoch 8/1000
Learning rate: 0.000001
2s for 12 steps - 148ms/step - loss 29.1965
Validation:
1s - loss 30.9551

Epoch 9/1000
Learning rate: 0.000001
2s for 12 steps - 147ms/step - loss 27.8498
Vali